# Variation methods


Main contribution of the paper:

- Methodological contribution of empirical robustness for _network_ epistemology. 

Key challenge:

- Test whether inequality (or some other network feature such as density, clustering, or diameter) increases reliability (or some other metric such as speed).
- Counterfactual: had the network been more equal, the group would have been more reliable.
- This requires us to identify networks that differ in their inequality but are otherwise maximally similar.
- We consider three network features: density, clustering, and degree inequality. So networks that have the similar density, clustering and degree inequality are considered maximally similar. (I left out diameter here.)

To achieve this, we develop some variation methods. The main goals for these variation methods are:

1. Control: Ability to tinker with specific network features (density, inequality, clustering)
2. Simplicity
3. Computational tractability
4. Link to (individualistic) intervention 

There are two types of variation methods: 

1. Possibly complex variation methods that can be used to produce networks with specific network properties (density, inequality, clustering), i.e., high control
2. Intuitive and simple variation methods that yield a lower degree of control over the specific network properties (density, inequality, clustering)

The basic ideas in these variation methods are the following:

1. Density
    - Increase by adding edges
    - Keep constant by rewiring edges
    - Decrease by removing edges [not implemented]
2. Clustering
    - Note: the local clustering coefficient of a given node basically is the number of triangles that pass by that node divided by the number of possible triangles that pass by that node. 
    - Increase by adding edges that create new triangles
    - Keep constant ???
    - Decrease by removing edges from existing triangles
3. Inequality
    - Decrease by adding edges randomly (following the uniform degree distribution)
    - Keep constant by adding edges following the original degree distribution (i.e., that of the PUD network)
    - Increase by (a) sequentially adding edges preferentially, or  (b) by adding edges following a degree distribution that is more unequal than the original one [neither implemented]

## Hop over to Outcomes section to see the results of the different variation methods

## Simple variation methods

### Equalizers: tinker with inequality

1. `randomize_network`: Randomly rewire edges (following the uniform degree distribution) [THIS IS WHAT WE CURRENTLY DO]
    - Density $=$
    - Clustering $\downarrow$
    - Inequality $\downarrow$
2. `equalize`: Rewire triangles: Take a triangle, take a node in the triangle, remove the edge in the triangle that does not contain the node, then add a random new edge that creates a new triangle that passed by the node. (The goal was to keep clustering somewhat equal, while increasing equality.)
    - Density $=$
    - Clustering $\approx \downarrow$
    - Inequality $\downarrow$

### Densify
1. [not implemented] Randomly add edges (following the uniform degree distribution)
    - Expectations
        - Density $\uparrow$
        - Clustering $\downarrow$
        - Inequality $\downarrow$
2. `densify`: Add edges following the original degree distribution
    - Density $\uparrow$
    - Clustering $\downarrow$
    - Inequality $=$
3. `cluster`: Add edges that create new triangles (taking into account the original degree distribution)
    - Density $\uparrow$
    - Clustering $\Uparrow$
    - Inequality $\approx$ 

### Clustering
1. `decluster`: Remove edges from existing triangles and add new edge following the original degree distribution
    - Density $=$
    - Clustering $\Downarrow$
    - Inequality $\approx\downarrow$


## Complex variation methods

1. `densify_fancy`: Add edges in such a way to attempt to reach a target clustering coefficient and a target degree distribution (original or uniform). 
    - Basically, the algorithm checks whether the target clustering coefficient has been reached. If not, it adds an edge that increases clustering. If yes, it adds a new edge following the target degree distribution. 
    - **Computationally costly**: after an edge is added, the algorithm calculates the clustering coefficient of the new network, which is computationally costly to an unacceptable degree (I think). 
    - **High degree of control**, especially regarandoming the clustering coefficient: For example, can be used to achieve the following:
        1. Only increase density
            - Density $\uparrow$
            - Clustering $=$
            - Inequality $=$
        2. Increase density and decrease inequality
            - Density $\uparrow$
            - Clustering $=$
            - Inequality $\downarrow$
2. `densify_fancy_speed_up`: This method basically works the same as `densify_fancy`, but has a significant speed up. Instead of calculating the average clustering coefficient whenever an edge is added, we only calculate the new local clusterinf coefficient for nodes that are affected by the newly added edge. 
    - **Computationally not cheap**: although the algorithm is much quicker than `densify_fancy`, it still takes some time. I expect that the computational costs are acceptable.
    - **High control**, especially regarandoming the clustering coefficient.
3. `densify_semi_fancy`: Add edges either to increase clustering or following the target degree distribution — with a fixed probability.
    - Basically, the algorithm throws a (biased) coin to determine whether it will add an edge that increases clustering or add one following the target degree distribution. 
    - Computationally fast. Relies on a biased coin toss instead of calculating the clustering coefficient. The drawback is that the clustering coefficient will change to some degree.
    - Pretty high degree of control.


- Note: we could keep the density fixed by first removing a number of edges and then using `densify_fancy` or `densify_semi_fancy` to add the same number of edges. 

## Thoughts and observations

- I was surprised to learn that both the in-degree and the out-degree distributions of the PUD network are scale free. However, they are not neatly correlated. Nonetheless, my suspicion is that there is a pattern between an agent’s number of publication and both its in-degree and its out-degree. 
- I was surprised to learn that the clustering coefficient is not based on directed graphs (`nx.average_clustering`). 
- In a sense, we have currently only implemented the target degree distribution of the degree distribution of the input network (the PUD network) or the uniform distribution. We could consider other degree distributions (i.e., an unequal distribution that flips the original degree distribution)
- 


## Setup

In [ ]:

from imports import *
from network_utils import *
# import random as random

network_pud: nx.DiGraph = dill.load(open('empirical_networks/pud_final.pkl', 'rb'))

## Randomize

In [ ]:
random_net_1 = randomize_network(network_pud, p_rewiring=0.1)
random_net_2 = randomize_network(network_pud, p_rewiring=0.2)
random_net_3 = randomize_network(network_pud, p_rewiring=0.3)
random_net_4 = randomize_network(network_pud, p_rewiring=0.4)
random_net_5 = randomize_network(network_pud, p_rewiring=0.5)

In [ ]:
data_random = {
    "network": ["original", "random_10%", "random_20%", "random_30%", "random_40%", "random_50%"],
    "degree_gini": [
        calculate_degree_gini(network_pud, directed=True),
        calculate_degree_gini(random_net_1, directed=True),
        calculate_degree_gini(random_net_2, directed=True),
        calculate_degree_gini(random_net_3, directed=True),
        calculate_degree_gini(random_net_4, directed=True),
        calculate_degree_gini(random_net_5, directed=True)
    ],
    "clustering_coefficient": [
        nx.average_clustering(network_pud),
        nx.average_clustering(random_net_1),
        nx.average_clustering(random_net_2),
        nx.average_clustering(random_net_3),
        nx.average_clustering(random_net_4),
        nx.average_clustering(random_net_5)
    ],
    "density": [
        nx.density(network_pud),
        nx.density(random_net_1),
        nx.density(random_net_2),
        nx.density(random_net_3),
        nx.density(random_net_4),
        nx.density(random_net_5)
    ]
}
df_random = pd.DataFrame(data_random)
df_random


## Equalize

In [ ]:
equalized_net_1 = equalize(network_pud, n=1000)
equalized_net_2 = equalize(network_pud, n=2000)
equalized_net_3 = equalize(network_pud, n=3000)
equalized_net_4 = equalize(network_pud, n=4000)
equalized_net_5 = equalize(network_pud, n=5000)


In [ ]:
data_equalize = {
    "network": ["original", "equalized_1000", "equalized_2000", "equalized_3000", "equalized_4000", "equalized_5000"],
    "degree_gini": [
        calculate_degree_gini(network_pud, directed=True),
        calculate_degree_gini(equalized_net_1, directed=True),
        calculate_degree_gini(equalized_net_2, directed=True),
        calculate_degree_gini(equalized_net_3, directed=True),
        calculate_degree_gini(equalized_net_4, directed=True),
        calculate_degree_gini(equalized_net_5, directed=True)
    ],
    "clustering_coefficient": [
        nx.average_clustering(network_pud),
        nx.average_clustering(equalized_net_1),
        nx.average_clustering(equalized_net_2),
        nx.average_clustering(equalized_net_3),
        nx.average_clustering(equalized_net_4),
        nx.average_clustering(equalized_net_5)
    ],
    "density": [
        nx.density(network_pud),
        nx.density(equalized_net_1),
        nx.density(equalized_net_2),
        nx.density(equalized_net_3),
        nx.density(equalized_net_4),
        nx.density(equalized_net_5)
    ]
}

df_equalize = pd.DataFrame(data_equalize)
df_equalize

## Densify

In [ ]:
densify_fancy_speedy_1000 = densify_fancy_speed_up(network_pud, n_edges=1000, target_degree_dist="original")
print('Done')
densify_fancy_speedy_2000 = densify_fancy_speed_up(network_pud, n_edges=2000, target_degree_dist="original")
print('Done')
densify_fancy_speedy_3000 = densify_fancy_speed_up(network_pud, n_edges=3000, target_degree_dist="original")
print('Done')
densify_fancy_speedy_4000 = densify_fancy_speed_up(network_pud, n_edges=4000, target_degree_dist="original")
print('Done')
densify_fancy_speedy_5000 = densify_fancy_speed_up(network_pud, n_edges=5000, target_degree_dist="original")

In [ ]:
data_densify_fancy = {
    "network": ["original", "densify_fancy_speedy_1000", "densify_fancy_speedy_2000", "densify_fancy_speedy_3000", "densify_fancy_speedy_4000", "densify_fancy_speedy_5000"], 
    "degree_gini": [
        calculate_degree_gini(network_pud, directed=True),
        calculate_degree_gini(densify_fancy_speedy_1000, directed=True),
        calculate_degree_gini(densify_fancy_speedy_2000, directed=True),
        calculate_degree_gini(densify_fancy_speedy_3000, directed=True),
        calculate_degree_gini(densify_fancy_speedy_4000, directed=True),
        calculate_degree_gini(densify_fancy_speedy_5000, directed=True)
    ],
    "clustering_coefficient": [
        nx.average_clustering(network_pud),
        nx.average_clustering(densify_fancy_speedy_1000),
        nx.average_clustering(densify_fancy_speedy_2000),
        nx.average_clustering(densify_fancy_speedy_3000),
        nx.average_clustering(densify_fancy_speedy_4000),
        nx.average_clustering(densify_fancy_speedy_5000)
    ],
    "density": [
        nx.density(network_pud),
        nx.density(densify_fancy_speedy_1000),
        nx.density(densify_fancy_speedy_2000),
        nx.density(densify_fancy_speedy_3000),
        nx.density(densify_fancy_speedy_4000),
        nx.density(densify_fancy_speedy_5000)
    ]
}

df_densify_fancy_speedy = pd.DataFrame(data_densify_fancy)
df_densify_fancy_speedy

## Cluster

In [ ]:
decluster_net_1 = decluster(network_pud, n_triangles=1000)
print("Done")
decluster_net_2 = decluster(network_pud, n_triangles=2000)
print("Done")
decluster_net_3 = decluster(network_pud, n_triangles=3000)    
print("Done")
decluster_net_4 = decluster(network_pud, n_triangles=4000)
print("Done")
decluster_net_5 = decluster(network_pud, n_triangles=5000)

In [ ]:
data_decluster = {
    "network": ["original", "declustered_1000", "declustered_2000", "declustered_3000", "declustered_4000", "declustered_5000"],
    "degree_gini": [
        calculate_degree_gini(network_pud, directed=True),
        calculate_degree_gini(decluster_net_1, directed=True),
        calculate_degree_gini(decluster_net_2, directed=True),
        calculate_degree_gini(decluster_net_3, directed=True),
        calculate_degree_gini(decluster_net_4, directed=True),
        calculate_degree_gini(decluster_net_5, directed=True)
    ],
    "clustering_coefficient": [
        nx.average_clustering(network_pud),
        nx.average_clustering(decluster_net_1),
        nx.average_clustering(decluster_net_2),
        nx.average_clustering(decluster_net_3),
        nx.average_clustering(decluster_net_4),
        nx.average_clustering(decluster_net_5)
    ],
    "density": [
        nx.density(network_pud),
        nx.density(decluster_net_1),
        nx.density(decluster_net_2),
        nx.density(decluster_net_3),
        nx.density(decluster_net_4),
        nx.density(decluster_net_5)
    ]
}

df_decluster = pd.DataFrame(data_decluster)
df_decluster

In [ ]:
cluster_network_1 = cluster_network(network_pud, n=1000)
print("Done")
cluster_network_2 = cluster_network(network_pud, n=2000)
print("Done")
cluster_network_3 = cluster_network(network_pud, n=3000)
print("Done")
cluster_network_4 = cluster_network(network_pud, n=4000)
print("Done")

In [ ]:
data_cluster = {
    "network": ["original", "clustered_1000", "clustered_2000", "clustered_3000", "clustered_4000"],
    "degree_gini": [
        calculate_degree_gini(network_pud, directed=True),
        calculate_degree_gini(cluster_network_1, directed=True),
        calculate_degree_gini(cluster_network_2, directed=True),
        calculate_degree_gini(cluster_network_3, directed=True),
        calculate_degree_gini(cluster_network_4, directed=True)
    ],
    "clustering_coefficient": [
        nx.average_clustering(network_pud),
        nx.average_clustering(cluster_network_1),
        nx.average_clustering(cluster_network_2),
        nx.average_clustering(cluster_network_3),
        nx.average_clustering(cluster_network_4)
    ],
    "density": [
        nx.density(network_pud),
        nx.density(cluster_network_1),
        nx.density(cluster_network_2),
        nx.density(cluster_network_3),
        nx.density(cluster_network_4)
    ]
}

df_cluster = pd.DataFrame(data_cluster)
df_cluster

## Outcomes

In [ ]:
df_random

In [ ]:
df_equalize

In [ ]:
df_densify_fancy_speedy

In [ ]:
df_decluster 

In [ ]:
df_cluster